In [74]:
import cv2
import os
import random
import numpy as np

def generate_random_polygon(image, min_area):
    height, width, _ = image.shape
    
    while True:
        point_1 = [np.random.randint(0, width), np.random.randint(0, height)]
        point_2 = [np.random.randint(0, width), np.random.randint(0, height)]
        point_3 = [np.random.randint(0, width), np.random.randint(0, height)]
        polygon = np.array([point_1, point_2, point_3], np.int32)
        
        # Gets area of triangle
        area = 0.5 * abs((point_1[0]*(point_2[1] - point_3[1])) + 
                         (point_2[0]*(point_3[1] - point_1[1])) + 
                         (point_3[0]*(point_1[1] - point_2[1])))
        
        if area >= min_area:
            return polygon.reshape((-1, 1, 2))
        
def adjust_contrast_brightness(image):
    
    alpha = random.uniform(0.5, 1.5)
    beta = random.randint(-50, 50)

    image_float = image.astype(np.float32)
    
    adjusted_image = cv2.multiply(image_float, np.array([alpha]))
    adjusted_image = cv2.add(adjusted_image, beta)
    adjusted_image = np.clip(adjusted_image, 0, 255)
    adjusted_image = adjusted_image.astype(np.uint8)
    
    return adjusted_image

def add_triangle(image, num_shadows, min_area, transparency):

    image_with_shadow = np.copy(image)
    
    for _ in range(num_shadows):

        temp_img = np.copy(image)
        polygon = generate_random_polygon(image, min_area)
        shadow_color = (np.random.randint(0, 255), np.random.randint(0, 255), np.random.randint(0, 255))  
        
        cv2.fillPoly(temp_img, [polygon], shadow_color)
        cv2.addWeighted(image_with_shadow, transparency, temp_img, transparency, 0, image_with_shadow)
    
    return image_with_shadow

def shear_image(image, shear_factor=0.0):

    rows, cols, _ = image.shape
    
    M = np.array([[1, shear_factor, 0],
                  [0, 1, 0]])
    
    sheared_img = cv2.warpAffine(image, M, (cols, rows))
    
    if shear_factor > 0:
        sheared_img = sheared_img[:, int(cols*shear_factor):]
    elif shear_factor < 0:
        sheared_img = sheared_img[:, :int(cols*(1+shear_factor))]
    
    return sheared_img

def process_image(image, min_area, num_shadows, shear_factor, poly_transparency):

    image = add_triangle(image, num_shadows=num_shadows, min_area=min_area, transparency=poly_transparency)
    image = adjust_contrast_brightness(image)
    image = shear_image(image, shear_factor=shear_factor)
    
    return image

def create_background():
    backgrounds_dir = "bg_base_imgs"
    backgrounds = os.listdir(backgrounds_dir)
    selected_background_filename = random.choice(backgrounds)
    print(selected_background_filename)
    selected_background_path = os.path.join(backgrounds_dir, selected_background_filename)
    background_img = cv2.imread(selected_background_path)

    processed_image = process_image(background_img, 5000, 7, 0.2, 0.5)

    return processed_image


In [75]:
def create_larger_canvas(image, canvas_size):
    # Create a larger blank canvas
    blank_canvas = np.zeros((canvas_size, canvas_size, 4), dtype=np.uint8)

    # Calculate the position to place the original image
    x_offset = (canvas_size - image.shape[1]) // 2
    y_offset = (canvas_size - image.shape[0]) // 2

    # Place the image in the center of the larger canvas
    blank_canvas[y_offset:y_offset+image.shape[0], x_offset:x_offset+image.shape[1]] = image

    return blank_canvas

def rotate_image(image, angle):
    # Load the image with alpha channel


    # Split the alpha channel
    bgr, alpha = image[..., :3], image[..., 3]

    # Get the image dimensions
    (h, w) = bgr.shape[:2]

    # Calculate the center of the image
    center = (w // 2, h // 2)

    # Calculate the size of the new bounding box
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    cos = np.abs(M[0, 0])
    sin = np.abs(M[0, 1])
    new_w = int((h * sin) + (w * cos))
    new_h = int((h * cos) + (w * sin))

    # Adjust the rotation matrix for the new bounding box
    M[0, 2] += (new_w / 2) - center[0]
    M[1, 2] += (new_h / 2) - center[1]

    # Perform the rotation
    rotated_bgr = cv2.warpAffine(bgr, M, (new_w, new_h))
    rotated_alpha = cv2.warpAffine(alpha, M, (new_w, new_h))

    # Merge the channels back
    rotated_image = cv2.merge([rotated_bgr, rotated_alpha])

    return rotated_image

def zoom_image(image, zoom_factor):
    # Calculate new dimensions
    new_w = int(image.shape[1] * zoom_factor)
    new_h = int(image.shape[0] * zoom_factor)

    # Resize the image
    resized_image = cv2.resize(image, (new_w, new_h), interpolation=cv2.INTER_LINEAR)

    return resized_image

def shear_card_image(image, shear_factor_x, shear_factor_y):
    # Get the image dimensions
    (h, w) = image.shape[:2]

    # Define the shearing matrix
    M = np.array([[1, shear_factor_x, 0],
                  [shear_factor_y, 1, 0]], dtype=float)

    # Apply the shearing transformation
    sheared_image = cv2.warpAffine(image, M, (w + int(shear_factor_x * h), h + int(shear_factor_y * w)))

    return sheared_image

def add_random_shadows(image):
    has_alpha = image.shape[2] == 4

    # Separate the alpha channel if present
    if has_alpha:
        alpha_channel = image[:, :, 3]
        image_rgb = image[:, :, :3]
    else:
        image_rgb = image

    # Create a single-channel shadow mask
    shadow_mask = np.zeros(image_rgb.shape[:2], dtype=np.uint8)

    # Randomly choose the number of shadows and their properties
    num_shadows = random.randint(3, 5)  # Increase the number of shadows
    for _ in range(num_shadows):
        x = random.randint(0, image_rgb.shape[1] // 4)  # More central x coordinate
        y = random.randint(0, image_rgb.shape[0] // 4)  # More central y coordinate
        w = random.randint(image_rgb.shape[1] // 4, image_rgb.shape[1] // 2)  # Larger width
        h = random.randint(image_rgb.shape[0] // 4, image_rgb.shape[0] // 2)  # Larger height

        # Adjust width and height to ensure the shadow is within the image boundaries
        w = min(w, image_rgb.shape[1] - x)
        h = min(h, image_rgb.shape[0] - y)

        intensity = random.randint(30, 180)

        # Create a linear gradient for the shadow
        gradient = np.linspace(intensity, 0, max(w, h)).astype(np.uint8)
        shadow = np.zeros((h, w), dtype=np.uint8)
        for i in range(h):
            shadow[i, :] = gradient[:w]

        # Place the shadow in the shadow mask
        shadow_mask[y:y+h, x:x+w] = shadow

    # Convert shadow mask to BGR
    shadow_mask_bgr = cv2.cvtColor(shadow_mask, cv2.COLOR_GRAY2BGR)

    # Blend the shadow mask with the original RGB image
    shadowed_image_rgb = cv2.addWeighted(image_rgb, 1, shadow_mask_bgr, -0.5, 0)

    # Re-attach the alpha channel if it was present
    if has_alpha:
        shadowed_image = cv2.merge((shadowed_image_rgb, alpha_channel))
    else:
        shadowed_image = shadowed_image_rgb

    return shadowed_image

def add_random_noise(image):
    # Get the image shape
    height, width, _ = image.shape

    # Create a noise matrix with Gaussian noise for RGB channels only
    noise = np.random.normal(0.05, .35, (height, width, 3)).astype(np.uint8)

    # Add the noise to the RGB channels while keeping alpha channel unchanged
    noisy_image = image.copy()
    noisy_image[:, :, :3] = np.clip(noisy_image[:, :, :3] + noise, 0, 255).astype(np.uint8)

    return noisy_image

def find_bounding_box_with_margin(image):
    alpha_channel = image[:, :, 3]

    # Threshold the alpha channel to get a binary image
    _, binary_alpha = cv2.threshold(alpha_channel, 1, 255, cv2.THRESH_BINARY)

    # Find contours from the binary alpha image
    contours, _ = cv2.findContours(binary_alpha, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # In case no contours are found, return the whole image as the bounding box
    if not contours:
        return 0, 0, image.shape[1], image.shape[0]

    # Find the largest contour which will be the contour of the card
    largest_contour = max(contours, key=cv2.contourArea)

    # Get the bounding box around the largest contour
    x, y, w, h = cv2.boundingRect(largest_contour)

    return x, y, x+w, y+h

def random_augmentation(image_path):
    # Load the image with alpha channel
    image = cv2.imread(image_path, cv2.IMREAD_UNCHANGED)

    # Create a larger canvas
    max_dimension = max(image.shape[:2])
    canvas_size = int(2.5 * max_dimension)  # Adjust the factor as needed
    image_on_canvas = create_larger_canvas(image, canvas_size)

    # Randomize transformations
    random_angle = random.uniform(-65, 65)  # Rotation between -45 and 45 degrees
    random_zoom = random.uniform(0.2, 0.5)  # Zoom between 0.4 and 0.9
    random_shear_x = random.uniform(-0.25, 0.25)  # Shear in X between -0.4 and 0.4
    random_shear_y = random.uniform(-0.25, 0.25)  # Shear in Y between -0.4 and 0.4

    # Apply transformations
    rotated_image = rotate_image(image_on_canvas, random_angle)
    zoomed_image = zoom_image(rotated_image, random_zoom)
    sheared_image = shear_card_image(zoomed_image, random_shear_x, random_shear_y)
    x0, y0, x1, y1 = find_bounding_box_with_margin(sheared_image)
    # Draw the bounding box on the image before cropping

    cropped_image = sheared_image[y0:y1, x0:x1]
    
    shadowed_image = add_random_shadows(cropped_image)
    final_image = add_random_noise(shadowed_image)

    # ... [rest of the code for finding bounding box, cropping, and saving] ...

    cv2.imwrite('augmented_image.png', final_image)

image_path = 'cards/2_of_clubs.png'
random_augmentation(image_path)


NameError: name 'random_shear_y' is not defined

In [ ]:
def overlay_card_on_background(card_image, background, min_width=150, max_width=170, margin=60, bbox_color=(0, 255, 0), bbox_thickness=2):
    # Ensure card image has an alpha channel
    if card_image.shape[2] != 4:
        raise ValueError("Card image must have an alpha channel.")

    # Background dimensions
    bg_height, bg_width, _ = background.shape

    # Card image dimensions and aspect ratio
    card_height, card_width, _ = card_image.shape
    aspect_ratio = card_height / card_width

    # Resize card
    new_width = np.random.randint(min_width, min(max_width, bg_width - 2 * margin))
    new_height = int(new_width * aspect_ratio)
    resized_card = cv2.resize(card_image, (new_width, new_height), interpolation=cv2.INTER_AREA)

    # Alpha channel for blending
    alpha_s = resized_card[:, :, 3] / 255.0
    alpha_l = 1.0 - alpha_s

    # Random position for the card (within margins)
    x_offset = np.random.randint(margin, bg_width - new_width - margin)
    y_offset = np.random.randint(margin, bg_height - new_height - margin)

    # Overlay the card on the background
    for c in range(3):
        background_slice = background[y_offset:y_offset + new_height, x_offset:x_offset + new_width, c]
        card_slice = resized_card[:, :, c]
        background[y_offset:y_offset + new_height, x_offset:x_offset + new_width, c] = \
            alpha_s * card_slice + alpha_l * background_slice
            
            
    top_left = (x_offset, y_offset)
    bottom_right = (x_offset + new_width, y_offset + new_height)
    cv2.rectangle(background, top_left, bottom_right, bbox_color, bbox_thickness)

    return background


background = create_background()
resized_background = cv2.resize(background, (500, 500))

# Generate augmented card image
random_augmentation('cards/2_of_clubs.png')  # Assuming this saves the image as 'augmented_image.png'
augmented_card = cv2.imread('augmented_image.png', cv2.IMREAD_UNCHANGED)

# Overlay the card on the background
final_image = overlay_card_on_background(augmented_card, resized_background)

# Display or save the final image
cv2.imshow('Augmented Card on Background', final_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

alina-rubo-RHa2IWIHH-w-unsplash.jpg
